In [1]:
import os
import pandas as pd
from relm.histogram import Histogram
from relm.mechanisms import SmallDB
import numpy as np
from itertools import product
import scipy.sparse as sps

In [2]:
data = pd.DataFrame()
test_results = np.random.choice(["Negative", "Positive", "N/A"], size=10000, p=[0.55,0.35,0.1]) 
data["Test Result"] = test_results

hist = Histogram(data)
real_database = hist.get_db()
db_size = real_database.size

In [3]:
from itertools import product

queries = [[{"Test Result": "Negative"}, {"Test Result": "Positive"}],
           [{"Test Result": "Positive"}, {"Test Result": "N/A"}],
           [{"Test Result": "Negative"}, {"Test Result": "N/A"}]]
queries = sps.vstack([hist.get_query_vector(q) for q in queries])

# queries = np.array([[1,1,0],
#                     [0,1,1],
#                     [1,0,1]])
# queries = sps.csr.csr_matrix(queries)

values = (queries @ real_database)/real_database.sum()

In [4]:
smalldb = SmallDB(epsilon=1.0, alpha=0.05)

In [5]:
synthetic_database = smalldb.release(values, queries, db_size)

In [6]:
TRIALS = 2**4
synthetic_responses = np.empty((TRIALS, queries.shape[0]))
for i in range(TRIALS):
    smalldb = SmallDB(epsilon=1.0, alpha=0.05)
    synthetic_database = smalldb.release(values, queries, db_size)
    synthetic_responses[i] = (queries @ synthetic_database) / synthetic_database.sum()

In [7]:
df = pd.DataFrame(data=np.row_stack((values, synthetic_responses)),
                  columns=["Query %i" % i for i in range(queries.shape[0])],
                  index=["Real Responses"] + ["TRIAL %i" % i for i in range(TRIALS)])

display(df)

,Query 0,Query 1,Query 2
Real Responses,0.900500,0.447700,0.651800
TRIAL 0,0.900833,0.445833,0.653333
TRIAL 1,0.900833,0.449167,0.650000
TRIAL 2,0.901667,0.447500,0.650833
TRIAL 3,0.901667,0.446667,0.651667
TRIAL 4,0.900833,0.447500,0.651667
TRIAL 5,0.899167,0.447500,0.653333
TRIAL 6,0.896667,0.451667,0.651667
TRIAL 7,0.901667,0.446667,0.651667
TRIAL 8,0.900833,0.447500,0.651667


In [8]:
print(db_size)

3
